In [ ]:
%matplotlib auto

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyro.infer
import torch
from numpy import exp

from tqdm import tqdm

from algorithms.hmc import HMC
from algorithms.sghmc import SGHMC
from algorithms.sgld import SGLD
from algorithms.sghmc_alt import SGHMCAlt


def potential_energy(x):
    return -2 * x ** 2 + x ** 4

def potential_grad(x):
    return - 4 * x + 4 * x ** 3

def noisy_grad(x):
    return potential_grad(x) + 2*np.random.normal(0,1)

def run_simulation(sample_count, initial_params, step_size, posterior, mass, do_mh):
    if do_mh:
        hmc = HMC(posterior, mass=mass, step_size=step_size, step_count=50, potential_energy=potential_energy)
    else:
        hmc = HMC(posterior, mass=mass, step_size=step_size, step_count=50)
    param = initial_params
    samples = [param]
    
    for _ in tqdm(range(sample_count)):
        param = hmc.sample(param)
        samples.append(param)
    return samples

def run_simulation_sghmc(sample_count, initial_params, step_size, posterior, mass, C, B):
    sghmc = SGHMC(posterior, mass=mass, step_size=step_size, step_count=5, friction_term=C, noise_model_estimate=lambda x: B*np.eye(initial_params.size))
    param = initial_params
    samples = [param]

    for _ in tqdm(range(sample_count)):
        param = sghmc.sample(param)
        samples.append(param)
    return samples

def run_simulation_sgld(sample_count, initial_params, posterior):    
    sgld = SGLD(eps_t, posterior)
    param = initial_params
    samples = [param]
    
    for _ in tqdm(range(sample_count)):
        param = sgld.sample(param)
        samples.append(param)
    return np.array(samples)


def plot_graph(x, y, step_size, label="",linestyle=None):
    y = y / (y.sum() * step_size)
    if linestyle==None:
        plt.plot(x, y, label=label)
    else:
        plt.plot(x, y, linestyle, label=label, lw=1)

In [ ]:
NUM_SAMPLES = 10000

samples_hmc_no_mh = run_simulation(
    sample_count=NUM_SAMPLES, initial_params=np.array([1.0]), step_size=0.1,
    posterior=potential_grad, mass=np.array([[1.0]]), do_mh=False)

samples_naive_no_mh = run_simulation(
    sample_count=NUM_SAMPLES, initial_params=np.array([1.0]), step_size=0.1,
    posterior=noisy_grad, mass=np.array([[1.0]]), do_mh=False)

samples_hmc_mh = run_simulation(
    sample_count=NUM_SAMPLES, initial_params=np.array([1.0]), step_size=0.1,
    posterior=potential_grad, mass=np.array([[1.0]]), do_mh=True)

samples_naive_mh = run_simulation(
    sample_count=NUM_SAMPLES, initial_params=np.array([1.0]), step_size=0.1,
    posterior=noisy_grad, mass=np.array([[1.0]]), do_mh=True)

samples_sghmc = run_simulation_sghmc(
    sample_count=NUM_SAMPLES, initial_params=np.array([1.0]), step_size=0.1,
    posterior=noisy_grad, mass=np.array([[1.0]]),
    C=np.array([[4]]), B=np.array([[0.2]]))

In [ ]:
grid_size = 0.1
grid = np.arange(-2, 2 + grid_size, grid_size)

plot_graph(grid, exp(-potential_energy(grid)), grid_size, label="True Distribution")

ys, xs = np.histogram(samples_sghmc, bins=grid)
xs = xs[:-1] + grid_size / 2
plot_graph(xs, ys, grid_size, label="SGHMC", linestyle="-g")

ys, xs = np.histogram(samples_naive_no_mh, bins=grid)
xs = xs[:-1] + grid_size / 2
plot_graph(xs, ys, grid_size, label="Naive SGHMC(no MH)", linestyle="-.k")

ys, xs = np.histogram(samples_naive_mh, bins=grid)
xs = xs[:-1] + grid_size / 2
ys = ys*0.02
plot_graph(xs, ys, grid_size, label="Naive SGHMC(With MH)", linestyle="-k")

ys, xs = np.histogram(samples_hmc_no_mh, bins=grid)
xs = xs[:-1] + grid_size / 2
plot_graph(xs, ys, grid_size, label="HMC(no MH)", linestyle="-b")

ys, xs = np.histogram(samples_hmc_mh, bins=grid)
xs = xs[:-1] + grid_size / 2
plot_graph(xs, ys, grid_size, label="HMC(with MH)", linestyle="-m")

plt.legend()
plt.show()

# Figure 2

In [ ]:
def potential_energy(x):
    return  (x ** 2)/2

def potential_grad(x):
    return x

def noisy_grad(x):
    return potential_grad(x) + 2*np.random.normal(0,1)

In [ ]:
noisy_hmc = HMC(noisy_grad, mass=np.array([[1]]), step_size=0.1, step_count=50)
param = np.array(1)
samples = []
momentum = []
for i in tqdm(range(300)):
    param = noisy_hmc.sample(param, resample_momentum=False)
    samples.append(param)
    momentum.append(noisy_hmc.momentum)
plt.scatter(samples, momentum, s=12, edgecolors='r', marker='^', facecolors='none', label="Noisy Hamiltonian Dynamics")

noisy_hmc = HMC(noisy_grad, mass=np.array([[1]]), step_size=0.1, step_count=50)
param = np.array(1)
samples = []
momentum = []
for i in tqdm(range(300)):
    param = noisy_hmc.sample(param, resample_momentum=(i%50==0))
    samples.append(param)
    momentum.append(noisy_hmc.momentum)
plt.scatter(samples, momentum, s=12, edgecolors='black', marker='x', facecolors='black', label="Noisy Hamiltonian Dynamics, resample every 50")

sghmc = SGHMC(noisy_grad, mass=np.array([[1]]), step_size=0.1, step_count=50, friction_term=np.array([[0.2]]), noise_model_estimate= lambda x: np.array([[0.2]]))
param = np.array(1)
samples = []
momentum = []
for i in tqdm(range(300)):
    param = sghmc.sample(param, resample_momentum=False)
    samples.append(param)
    momentum.append(sghmc.momentum)
plt.scatter(samples, momentum, s=12, edgecolors='lime', marker='o', facecolors='none', label="Noisy Hamiltonian Dynamics with Friction")

hmc = HMC(potential_grad, mass=np.array([[1]]), step_size=0.1, step_count=50)
param = np.array(1)
samples = []
momentum = []
for i in tqdm(range(300)):
    param = hmc.sample(param, resample_momentum=False)
    samples.append(param)
    momentum.append(hmc.momentum)
plt.scatter(samples, momentum, s=12, edgecolors='blue', marker='*', facecolors='none', label="Hamiltonian Dynamics")
plt.xlim(-10,10)
plt.ylim(-10,10)
plt.legend()
plt.show()

# Figure 3

In [ ]:
import seaborn as sns

In [ ]:
cov = np.array([[1,0.9],[0.9,1]])
invcov = np.linalg.inv(cov)
def potential_grad(theta):
    return invcov.dot(theta)
def noisy_grad(theta):
    return potential_grad(theta) + np.random.normal(0, 1, 2)

true_noise = lambda x: np.array([[1, 0.9], [0.9, 1]])

In [ ]:
underlying = np.random.multivariate_normal(mean=np.zeros(2), cov=cov, size=10000)
def plot_2d(samples):
    sns.jointplot(samples[:,0], samples[:,1], kind='kde')
plot_2d(underlying)

In [ ]:
posterior=noisy_grad
def sample_sgld(step_size, n_samples):
    sgld = SGLD(lambda t: (0.1*step_size) + (0.9*step_size)*(1-(t/n_samples)), posterior)
    param = np.zeros(2)
    samples_sgld = [param]
    for _ in tqdm(range(n_samples)):
            param = sgld.sample(param)
            samples_sgld.append(param)
    samples_sgld = np.array(samples_sgld)
    return samples_sgld

def sample_sghmc_alt(alpha, beta, eta, n_samples):
    sghmc = SGHMCAlt(posterior, np.identity(2), alpha, beta, eta)
    param = np.zeros(2)
    samples_sghmc = [param]      
    for _ in tqdm(range(n_samples)):
            param = sghmc.sample(param)
            samples_sghmc.append(param)
    samples_sghmc = np.array(samples_sghmc)
    return samples_sghmc

def sample_sghmc(step_size, noise_est, n_samples):
    noise = noise_est*step_size/2
    sghmc = SGHMC(posterior, mass=np.identity(2), step_size=step_size, step_count=1, friction_term=2*noise*np.eye(2), noise_model_estimate=lambda x: noise*np.eye(2))
    param = np.zeros(2)
    samples_sghmc = [param]      
    for _ in tqdm(range(n_samples)):
            param = sghmc.sample(param, resample_momentum=False)
            samples_sghmc.append(param)
    samples_sghmc = np.array(samples_sghmc)
    return samples_sghmc

## Figure 3a

In [ ]:
def autocorrelation_time(x):
    L, m = x.shape
    tau = []
    for i in range(m):
        unnorm = np.correlate(x[:,i], x[:,i], mode='full')
        acorr = unnorm[L-1:] / unnorm[L-1]
        res = 1
        for j in range(L):
            rpho = acorr[j]
            if abs(rpho) < 0.05:
                break
            res = res + rpho
        tau.append(res)
    return np.mean(tau)

In [ ]:
N_SAMPLES = 200000
true_cov = np.array([[1, 0.9], 
                     [0.9, 1]])
res_sgld = []

etaSGLD = 0.22

BURN_IN = 10000
for i in range(6):
    step = etaSGLD * (0.8**(i)) 
    samples_sgld = sample_sgld(step, N_SAMPLES)[BURN_IN:]
    cov_est_sgld = np.cov(samples_sgld, rowvar=False)
    cov_err_sgld = np.sum(np.absolute(cov_est_sgld-true_cov))
    acorrt_sgld = autocorrelation_time(samples_sgld)
    res_sgld.append((cov_err_sgld, acorrt_sgld))

In [ ]:
res_sgld

In [ ]:
N_SAMPLES = 300000
N_REPEATS = 1
res_sghmc = []
true_cov = np.array([[1, 0.9], 
                     [0.9, 1]])
BURN_IN = 10000
etaSGHMC = 0.1;
for step in [0.15,0.125,0.1,0.075,0.05]:
    mean_cov_est_sghmc = 0
    acorrt_sghmc = 0
    for _ in range(N_REPEATS):
        samples_sghmc = sample_sghmc(step, 1, int(N_SAMPLES))[BURN_IN:]
        cov_est_sghmc = np.cov(samples_sghmc, rowvar=False) 
        print(cov_est_sghmc)
        mean_cov_est_sghmc += cov_est_sghmc/N_REPEATS
        acorrt_sghmc += autocorrelation_time(samples_sghmc)/N_REPEATS
    res_sghmc.append((np.sum(np.absolute(mean_cov_est_sghmc-true_cov)), acorrt_sghmc))

In [ ]:
res_sghmc

In [ ]:
plt.plot([x[1] for x in res_sghmc], [x[0] for x in res_sghmc], label="SGHMC")
plt.plot([x[1] for x in res_sgld], [x[0] for x in res_sgld], label="SGLD")
plt.xlabel("Autocorrelation Time")
plt.ylabel("Covariance Est. Error")
plt.legend()
plt.show()

## Figure 3b

In [ ]:
samples_sgld = sample_sgld(0.1, 50)
samples_sghmc = sample_sghmc(0.1, 50)

sns.kdeplot(underlying[:,0], underlying[:,1], n_levels=7, cmap='hsv')
plt.scatter(samples_sgld[:,0], samples_sgld[:,1], marker='x', label='SGLD')
plt.scatter(samples_sghmc[:,0], samples_sghmc[:,1], marker='o', facecolors='none', edgecolors='red', label='SGHMC')
plt.legend()
plt.xlim(-2, 2)
plt.ylim(-2, 2.5)
plt.show()

# Exploring Friction

In [ ]:
from algorithms.dist_divergence import DistDivergence
import math
def very_noisy_grad(x):
    return potential_grad(x) + np.random.normal(0,math.sqrt(20))


log_volume = 1.67992624289378653348
div_est = DistDivergence(lambda x: -potential_energy(x)-log_volume)

In [ ]:
klds = []
NUM_SAMPLES = 15000
for friction in range(0,20):
    print(friction)
    samples_sghmc = run_simulation_sghmc(
        sample_count=NUM_SAMPLES, initial_params=np.array([1.0]), step_size=0.1,
        posterior=very_noisy_grad, mass=np.array([[1.0]]),
        C=np.array([[friction]]), B=np.array([[0]]))
    klds.append(div_est.kld(samples_sghmc))

In [ ]:
klds_correct = []
NUM_SAMPLES = 10000
for friction in range(1,20):
    print(friction)
    samples_sghmc = run_simulation_sghmc(
        sample_count=NUM_SAMPLES, initial_params=np.array([1.0]), step_size=0.1,
        posterior=very_noisy_grad, mass=np.array([[1.0]]),
        C=np.array([[friction]]), B=np.array([[1]]))
    klds_correct.append(div_est.kld(samples_sghmc))

In [ ]:
plt.plot(range(1,20), klds[1:], label="B^=0")
plt.plot(range(1,20), klds_correct, label="B^=B")
plt.xlabel("C")
plt.ylabel("KL Divergence")
plt.title("Bimodal Quartic")
plt.legend()
plt.show()

## Himmelblau

In [ ]:
import experiments.synthetic_3d as s3d
val, grad = s3d.himmelblau()
noisy_grad = lambda x: grad(x) + np.random.normal(0, math.sqrt(20), 2)
div_est_himmel = DistDivergence(lambda x: -val(x)-3.6590944696232746)

In [ ]:
klds = []
klds_correct = []
NUM_SAMPLES = 100000
for friction in range(2,20,2):
    print(friction)
    samples_sghmc = run_simulation_sghmc(
        sample_count=NUM_SAMPLES, initial_params=np.array([1.0,1.0]), step_size=0.1,
        posterior=noisy_grad, mass=np.eye(2),
        C=friction*np.eye(2), B=0*np.eye(2))
    klds.append(div_est_himmel.kld(samples_sghmc))
    samples_sghmc = run_simulation_sghmc(
        sample_count=NUM_SAMPLES, initial_params=np.array([1.0,1.0]), step_size=0.1,
        posterior=noisy_grad, mass=np.eye(2),
        C=friction*np.eye(2), B=np.eye(2))
    klds_correct.append(div_est_himmel.kld(samples_sghmc))

In [ ]:
plt.plot(range(2,20,2), klds, label="B^=0")
plt.plot(range(2,20,2), klds_correct, label="B^=B")
plt.xlabel("C")
plt.ylabel("KL Divergence")
plt.title("Himmelblau")
plt.legend()
plt.show()

# Rosenbrock

In [ ]:
import experiments.synthetic_3d as s3d
val, grad = s3d.rosenbrock()
noisy_grad = lambda x: grad(x) + np.random.normal(0, math.sqrt(20), 2)
div_est_rosen = DistDivergence(lambda x: -val(x)-3.7157366201462643)

In [ ]:
klds = []
klds_correct = []
NUM_SAMPLES = 100000
for friction in range(2,20,2):
    print(friction)
    samples_sghmc = run_simulation_sghmc(
        sample_count=NUM_SAMPLES, initial_params=np.array([1.0,1.0]), step_size=0.1,
        posterior=noisy_grad, mass=np.eye(2),
        C=friction*np.eye(2), B=0*np.eye(2))
    klds.append(div_est_rosen.kld(samples_sghmc))
    samples_sghmc = run_simulation_sghmc(
        sample_count=NUM_SAMPLES, initial_params=np.array([1.0,1.0]), step_size=0.1,
        posterior=noisy_grad, mass=np.eye(2),
        C=friction*np.eye(2), B=np.eye(2))
    klds_correct.append(div_est_rosen.kld(samples_sghmc))

In [ ]:
plt.plot(range(2,20,2), klds, label="B^=0")
plt.plot(range(2,20,2), klds_correct, label="B^=B")
plt.xlabel("C")
plt.ylabel("KL Divergence")
plt.title("Rosenbrock")
plt.legend()
plt.show()